# Gaussian Processes & Environmental Data

<hr>

**Why Environmental Data?**<br>

By using data such as air/water quality, temperature and other measurements, we can do the following:

- Understand underlying processes, changes (e.g. climate change, *'statistics of weather over time'*)
- Impacts on environment, health, economics, society
- Shape policies
- Forecast events, warnings (*e.g. seismic network, storms, ...*)
- Resource/energy management, *e.g. water, renewable energies*
- Use in planning, routing, backtracking, control

The underlying questions and techniques are easily transferrable to other domains. Some of the questions asked are:

- Relationships (correlations, association) in temporal, spatial dimensions
- trends; forecasting
- planning (*e.g. where to place more sensors for better measurements*)
- quantifying uncertainty, adaptive sensing

For example, in environmental data, we'll be interested in some of these questions:

- Predicting animal populations: Usually, animal populations serve as indicator of healthy ecosystems.

- Predicting disease: Environmental data can provide contextual information about disease spreading. For example, how temperature changes and rainy seasons impact the spread of malaria in the tropics.

- Renewable energy predictions: Clear understanding of wind speeds, water levels in damps, or underground thermal activity provide information for renewable energy decisions.

- Predicting extreme events: extreme events like storms and floods are usually modeled and predicted to incorporate action plans.

- Policy making: for example pollution concentrations measurements in the air can help the design of guidelines for industry contaminants. 

Examples of modeling flow data:

- Forward prediction: simulate (propogate) distributions, including variation in time
- Hindcasting (backtracking): where did this object initially come from?

<img alt="Flow Example" src="assets/flow_example.png" width="300">

*Example: Malaysian Airlines Flight 370*

<img alt="MH370" src="assets/MH370.png" width="300">

<hr>

**Spatial Correlation**

Typically metrics are highly correlated when pair distances are small. For example, temperature patterns between two relatively close locations would be likely similar. 

$\therefore$ Intuition: Correlation is a function of distance. How do we then estimate metrics in other locations?

Given temperature (*or other measurements*) of a particular location and the covariance matrix between a pair of locations, how do we derive the conditional distribution of the temperature at another location?

<img alt="Conditional Distribution of a Multivariate Normal" src="assets/conditional_multivariate_normal.jpg" width="300">

Model each location as a Gaussian random variable and formulate a multivariate Gaussian, $X \sim N(\mu_k, \Sigma_k)$

The PDF of a multivariate Gaussian RV, $p(x | \mu, \Sigma) = \frac{1}{(2\pi^{n/2}) \lvert \Sigma \rvert^{1/2}} \exp (-\frac{1}{2} (x - \mu)^T \Sigma^{-1} (x - \mu))$

where $\mu = \mathbb {E}[X]$, $\lvert \Sigma \rvert^{1/2}$ denotes the determinant of the covariance matrix $\Sigma$

Properties of the covariance operator:
- $\textsf{Cov}(X,X) = \sigma _ X^2$
- $\textsf{Cov}(aX + bY,cW +dV) = ac*\textsf{Cov}(X,W)+ad*\textsf{Cov}(X,V)+bc*\textsf{Cov}(Y,W)+bd*\textsf{Cov}(Y,V)$

A 2-dimensional Multivariate Gaussian PDF as an example:

$
\displaystyle  p\left( \begin{bmatrix}  {x}_1 \\ {x}_2 \end{bmatrix} \mid \begin{bmatrix}  \mu _1\\ \mu _2 \end{bmatrix} , \begin{bmatrix}  {\sigma }_{11} &  {\sigma }_{12} \\ {\sigma }_{21} &  {\sigma }_{22} \end{bmatrix} \right) =\frac{1}{2\pi ( {\sigma }_{11}{\sigma }_{22} -{\sigma }_{12}{\sigma }_{21} )^{1/2}} \exp \left(-\frac{1}{2}\left( \begin{bmatrix}  {x}_1 \\ {x}_2 \end{bmatrix} - \begin{bmatrix}  \mu _1\\ \mu _2 \end{bmatrix} \right) ^\intercal \begin{bmatrix}  {\sigma }_{11} &  {\sigma }_{12} \\ {\sigma }_{21} &  {\sigma }_{22} \end{bmatrix} ^{-1}\left( \begin{bmatrix}  {x}_1 \\ {x}_2 \end{bmatrix} - \begin{bmatrix}  \mu _1\\ \mu _2 \end{bmatrix} \right)\right).
$


The conditional expectation of $\mu_{A|B}$ is equal to $\mu_A + \frac{\sigma_{AB}}{\sigma_B^2} (y_B - \mu_B)$ and $\therefore$ the conditional expectation shifts by the correlation between the pair

The conditional variance reduces given that we know a related variable which gives us more information about the estimate, $\sigma_{A|B}^2 = \sigma_A^2 - \sigma_{AB} \sigma_{B}^{-2} \sigma_{AB}$

****

*Checkpoint: Look at `05b-spatial-prediction.ipynb` before proceeding further*

****
**Gaussian Process: Setup**

Given a pair of multivariate Gaussian random variables, $X_1 \in \mathbb{R}^d$, and $X_2 \in \mathbb{R}^{N-d}$ where $X_1$ are cities where we do not have measurements and $X_2$ are cities where we have measurements. $\mu_1$ and $\mu_2$ are available.

The random variables associated with physical locations represented by the variables $Z_1 \in \mathbb{R}^{M \times d}$ and $Z_2 \in \mathbb{R}^{M \times (N-d)}$ where $M = 2$ is typical for spatial data.

Given a covariance function $k(z_i, z_j)$ that serves as a proxy for the relationship between two random variables as a function of their spatial locations. The kernel function is used to construct a covariance matrix, $\Sigma_{ij} = cov(X_i, X_j) = k(z_i, z_j)$

$\displaystyle  \mathbf{\Sigma } = \begin{bmatrix}  \mathbf{\Sigma }_{11} \in \mathbb {R}^{d\times d} &  \mathbf{\Sigma }_{12} \in \mathbb {R}^{d \times (N-d)}\\ \mathbf{\Sigma }_{21} \in \mathbb {R}^{(N-d) \times d} &  \mathbf{\Sigma }_{22} \in \mathbb {R}^{(N-d) \times (N-d)} \end{bmatrix}$

The distribution of the random variable $X_1$ conditioned on $X_2 = x_2$ is a normal distribution with:

$\mu_{X_1 \vert X_2} = \mu_1 + \Sigma_{12} \Sigma_{22}^{-1} (x_2 - \mu_2)$

$\Sigma_{X_1 \vert X_2} = \Sigma_{11} - \Sigma_{12} \Sigma_{22}^{-1} \Sigma_{21}$

****

**Model Selection in Gaussian Processes**

Two ways to select kernel functions:

1. Estimate generalization error with cross-validation, leave-one-out or k-fold

    Given data $(x_1, y_1), \dots, (x_N, y_N)$:
    - For each $i$, remove $(x_i, y_i)$ from the data and fit a Gaussian Process to the rest (*leave-one-out*)
    - Test the fitted Gaussian Process on the removed data point, predict $\mu_{*}$, $\sigma_{*}$
    - Repeat for all $i$ data points and compute the average probability for a given $\theta$
    - Maximize $\log p(y_i | X, y, \theta) = -\frac{1}{2} \log \sigma_{*}^2 - \frac{(y_i - \mu_{*})^2}{2\sigma{*}^2} - \frac{1}{2} \log 2 \pi$
    - Choose $\theta$ such  that it maximizes the log predictive probability
    
        $\max \sum_{i=1}^{N} \log p(y_i | X, y_{-i}, \theta)$
        

2. Maximize log likelihood of the data, $p(y | X, \theta)$

    - Trades off data fit and complexity
    
        $\log p(y | X, \theta) = - \frac{1}{2} y^T K^{-1} y  - \frac{1}{2} \log \vert K \vert - \frac{n}{2} \log 2 \pi$
        
        where 
        
        - $\vert K \vert$ is the determinant of the covariance matrix
        - $- \frac{1}{2} y^T K^{-1} y$ is the data fit
        - $- \frac{1}{2} \log \vert K \vert$ is the complexity penalty
        
    <img alt="Model Selection" src="assets/model_selection.jpg" width="600">
    
    <img alt="Two Parameter Model Selection" src="assets/two_parameter_selection.jpg" width="600">

****

**Long-range Climate Correlations**


<hr>

# Basic code
A `minimal, reproducible example`

_Question 1_

Assume the temperature in City 1 $X_1$ is a Gaussian random variable with mean $\mu_1 = 60$ and $\sigma_1 = 10$, and that the temperature of City 2 is $X_2$ is a Gaussian random variable with mean $\mu_2 = 90$ and $\sigma_2 = 20$. Moreover, we know that the covariance between $X_1$ and $X_2$ is 100. Today, we have observed that the temperature in City 2 is 75. What is the probability that the new temperature in the City 1 is bigger than 56.25?

In [7]:
# Compute conditional expectation
mu_1, sigma_1 = 60, 10
mu_2, sigma_2 = 90, 20
cov = 100
observed_2 = 75

conditional_mu = mu_1 + cov/(sigma_2**2) * (observed_2 - mu_2)
print(conditional_mu)

56.25
